In [2]:
def data_cleansing(dataset):
    order_level_dataset = pd.DataFrame()
    group_col = dataset[
        ["order_number",
        "store_level_promotion_code",
        "mall_level_promotion_code",
        "gift_promotion_id"]
    ] # convert the promotion materials to True/False value then take a summation to represent the frequency of using promotion materials
    order_level_dataset['enjoyed_promotion'] = group_col.groupby("order_number").nunique().sum(axis=1)
    dataset.loc[:, "package_volume"] = dataset[
        ["height", "width", "length", "quantity"]
    ].product(axis=1) # convert the package size of product to volume
    dataset.loc[:, "total_weight"] = dataset[
        ["weight", "quantity"]
    ].product(axis=1) # calculate the package total weight
    group_col = dataset[["order_number", "package_volume", "total_weight"]]
    order_level_dataset[["total_package_volume", "total_package_weight"]] = group_col.groupby("order_number").sum()

    dummies_col = [
        "primary_category_name_en",
        "sub_cat_1_name_en",
        "delivery_information_en",
        "packing_box_type",
        "storage_type"
    ] # those are the target column to be dummy

    for col in tqdm.tqdm(dummies_col):
        dummy_df = pd.concat(
            [
                dataset.order_number,
                pd.get_dummies(dataset[col],prefix=col, dummy_na=True)
            ],axis=1
        )
        order_level_dataset = order_level_dataset.merge(
            dummy_df.groupby('order_number').sum(),
            left_index=True, right_index=True
        )

    group = dataset.groupby("order_number")
    order_level_dataset["items_purchased"] = group.primary_sku_id.nunique()
    order_level_dataset["prim_cat_variety"] = group.primary_category_name_en.nunique()
    order_level_dataset['sub_cat_1_variety'] = group.sub_cat_1_name_en.nunique()
    order_level_dataset['sub_cat_2_variety'] = group.sub_cat_2_name_en.nunique()
    order_level_dataset['sub_cat_3_variety'] = group.sub_cat_3_name_en.nunique()
    order_level_dataset['sub_cat_4_variety'] = group.sub_cat_4_name_en.nunique() # calculate the customer how vary on purchasing different categories
    order_level_dataset['items_under_promotion'] = group.sku_level_promotion_code.nunique() # different from the promotion material which is the event level promotion, the items under promotion is a sku level promotion
    order_level_dataset['production_region_variety'] = group.manufacturer_country_en.nunique()
    return order_level_dataset.reset_index()

In [7]:
import pandas as pd
import tqdm
raw_data = r"C:\Users\joech\OneDrive\workspace\HKTV OpenDataBank\RFM_project\official_draft\dataset\01_raw.csv"
lots_data = pd.read_csv(raw_data, low_memory = False, chunksize = 100000)
target_columns = [
    "primary_sku_id",
    "primary_category_name_en",
    "sub_cat_1_name_en",
    "sub_cat_2_name_en",
    "sub_cat_3_name_en",
    "sub_cat_4_name_en",
    "sku_level_promotion_code",
    "store_level_promotion_code",
    "mall_level_promotion_code",
    "gift_promotion_id",
    "delivery_information_en",
    "packing_box_type",
    "storage_type",
    "weight",
    "height",
    "width",
    "length",
    "quantity",
    "manufacturer_country_en"
]
dataset = pd.concat(
    [df.drop(target_columns, axis=1) for df in lots_data]
)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5760858 entries, 0 to 5760857
Data columns (total 26 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   hktv_ad_id                        object 
 1   membership_level                  object 
 2   order_number                      object 
 3   additional_parent_order           object 
 4   order_date                        object 
 5   order_time_range                  object 
 6   delivery_date                     object 
 7   delivery_timeslot                 object 
 8   app_type                          object 
 9   device_type                       object 
 10  sales_application                 object 
 11  payment_gateway                   object 
 12  area                              object 
 13  district                          object 
 14  housing_type                      object 
 15  order_value                       float64
 16  total_discounts                   fl

In [ ]:
data.to_csv(r"C:\Users\joech\OneDrive\workspace\HKTV OpenDataBank\RFM_project\official_draft\dataset\order_level_aggregation.csv")
